<a href="https://colab.research.google.com/github/LeeSeungwon89/Deep-learning_Theory/blob/main/7-2%20%EC%8B%AC%EC%B8%B5%20%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **7-2 심층 신경망**

인공 신경층에 여러 층을 추가하여 심층 신경망을 생성하겠습니다. 일반 신경망의 성능을 제고하는 작업입니다.

## **2개의 층**

이전 챕터에서 사용했던 패션 MNIST 데이터 세트를 가져오겠습니다. 픽셀 값을 0 ~ 1 사이로 좁히고, 2차원 배열을 1차원 배열로 펼친 후 훈련 세트에서 검증 세트를 분리하겠습니다.

In [1]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

4431872/4422102 [==============================] - 0s 0us/step


**심층 신경망(deep neural network, DNN)**을 생성하기 위해 기존 신경망 모델에 층 2개를 추가해 보겠습니다. 입력층과 출력층 사이에 새로운 밀집층 2개를 추가하는 것입니다. 추가할 밀집층은 **은닉층(hidden layer)**이라고 부릅니다.

출력층의 활성화 함수는 적용할 수 있는 종류가 제한됩니다. 이진 분류의 경우 시그모이드 함수를 적용하고, 다중 분류의 경우 소프트맥스 함수를 적용합니다. 반면 은닉층의 활성화 함수는 자유롭게 적용할 수 있습니다. 참고로 회귀 문제에서는 활성화 함수를 적용할 필요가 없습니다(`Dense` 클래스의 `activation` 매개변수에 값을 지정하지 않습니다). 회귀 출력은 임의의 숫자이므로 출력층의 선형 방정식에 따라 도출된 값을 그대로 출력하기 때문입니다.

은닉층에 많이 사용하는 함수는 시그모이드 함수입니다. 시그모이드 함수는 유닛의 출력값을 0 ~ 1 사이로 압축합니다. 시그모이드 활성 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 생성해 보겠습니다.

In [2]:
# 신경망의 첫 번째 층(여기서는 은닉층)에는 `input_shape` 매개변수에 입력 크기를 지정합니다.
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
# 출력층을 생성합니다.
dense2 = keras.layers.Dense(10, activation='softmax')

은닉층에 지정한 유닛은 100개지만 특별한 기준을 토대로 지정한 것은 아닙니다. 유닛 개수에 따라 성능이 좌우되므로 적합한 유닛 개수를 지정하는 일에는 상당한 노하우가 필요합니다. 다만 한 가지 조건이 있습니다. 출력층의 유닛보다 많은 유닛을 지정해야 합니다. 출력층의 유닛이 10개인데 은닉층에 이보다 더 적은 유닛을 지정하면 부족한 정보가 전달될 것입니다.

## **심층 신경망 만들기**

`Dense` 클래스로 생성한 두 인스턴스를 사용하여 심층 신경망을 생성해 보겠습니다.

In [3]:
# 여러 층을 추가하려면 리스트에 담아 지정합니다. 단, 출력층은 마지막 원소로 전달합니다. 
model = keras.Sequential([dense1, dense2])

모델의 `summary()` 메서드를 호출하여 각 층에 대한 정보를 확인해 보겠습니다.

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


은닉층과 출력층이 순차로 나열되었습니다. 은닉층의 출력 크기인 `(None, 100)`에서 첫 번째 차원인 `None`은 샘플 개수를 의미합니다. 아직 샘플 개수가 정의되지 않았으므로 `None`입니다. `fit()` 메서드에 훈련 데이터를 주입하면 이 데이터를 한꺼번에 사용하지 않고 나눠서 수차례에 걸쳐 경사 하강법(**미니배치 경사 하강법**)을 수행합니다. 케라스의 미니배치 크기는 기본적으로 32개이며 `fit()` 메서드의 `batch_size` 매개변수에 지정하여 크기를 변경할 수 있습니다. 따라서 샘플 개수를 `None`으로 두고 어떤 배치 크기에도 유연하게 대응하게 하는 것입니다. 이렇게 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원을 **배치 차원**이라고 부릅니다. 두 번째 차원인 `100`은 유닛 개수입니다.

`Param #`은 모델 파라미터 개수를 의미합니다. 각 층의 모델 파라미터 개수에 대한 산출 공식은 아래와 같습니다. 참고로 각 유닛 개수와 절편 개수는 동수입니다.

- `dense`: 입력층 픽셀 784개 x 은닉층 유닛 100개 + 절편 100개 = 78,500개

- `dense_1`: 은닉층 유닛 100개 x 출력층 유닛 10개 x 절편 10개 = 1,010개

총 모델 파라미터 개수와 훈련될 파라미터 개수는 79,510개로 동수입니다. 경사 하강법으로 훈련되지 않는 파라미터를 가진 층도 있으므로 `Non-trainable params`는 0입니다. 

## **층을 추가하는 다른 방법**

---

작업 중

---

## **렐루 함수**

## **옵티마이저**